In [ ]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
# vim:fenc=utf-8
"""
Twitter API Streaming
By David White
"""

__version__ = '0.1.0'

required = ['requests', 'pandas', 'os', 'json', 'time']

# Adds the token to the local environment
import os
os.environ['BEARER_TOKEN'] = 'YOUR BEARER TOKEN'

import pandas as pd
# Create a CSV file on the local system to store the data
store = pd.DataFrame(columns = ['conversation_id','created_at','author_id','lang','text','name','username','geo.place_id'])
store.info()
store.to_csv('store.csv')
bearer_token = os.environ.get("BEARER_TOKEN")

#Uncomment to verify the bearer token
#print(bearer_token)

def connect_to_twitter():
    bearer_token = os.environ.get("BEARER_TOKEN")
    return {"Authorization": "Bearer {}".format(bearer_token)}
headers = connect_to_twitter()

while(True):
    import time
    #print('I am Alive!'+'/n')

    def make_request(headers):
        import requests
        import json
        url = "https://api.twitter.com/2/tweets/search/recent"
        query_params = {'query': 'xinjiang',
                        'max_results': 10,
                        'expansions': 'author_id,geo.place_id',
                        'tweet.fields': 'created_at,lang'
                        #'user.fields': 'username'
                        }
        return requests.request("GET", url, params=query_params,    headers=headers).json()
    response = make_request(headers)
    #print(response)

    tweet_data = response['data']
    tweet_data = pd.json_normalize(tweet_data)
    tweet_data.rename(columns = {'id':'conversation_id'}, inplace = True)
    tweet_data.rename(columns = {'id':'conversation_id'}, inplace = True)
    if 'geo.place_id' in tweet_data.columns:
            print(tweet_data)
            tweet_data_reorder = tweet_data.reindex(columns=['conversation_id','created_at','author_id','lang','text','geo.place_id'])
            tweet_data_reorder.info()
    else:
            tweet_data.loc[:,'geo.place_id'] = ''
            #print(tweet_data_reorder)
            tweet_data_reorder = tweet_data.reindex(columns=['conversation_id','created_at','author_id','lang','text','geo.place_id'])
            tweet_data_reorder.info()
    #print(tweet_data)
    #tweet_data['conversation_id'] = tweet_data['id']
 
    # 

    tweet_user = response['includes']
    tweet_user = pd.json_normalize(tweet_user,record_path=['users'])
    tweet_user.rename(columns = {'id':'author_id'}, inplace = True)
    tweet_user.reindex(columns=['author_id','name','username'])
    #tweet_user['author_id'] = tweet_user['id']
    
    tweet_user.info()
    #print(tweet_user)

    result = pd.merge(tweet_data_reorder, tweet_user.rename(columns={'id':'author_id'}), on='author_id',  how='left')
    result_reorder = result[['conversation_id','created_at','author_id','lang','text','name','username','geo.place_id']]

    result_reorder.info()

    result_reorder.to_csv('store.csv', mode='a', index=True, header=False)

    time.sleep(90)